In [6]:
#Data For DeepLabV3
import os
import cv2
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

data_path = './'
data_list=["train","val","test"]

In [3]:
# 데이터 증폭 설정
data_gen_args = dict(
    rotation_range=360,         # 랜덤 회전 범위
    width_shift_range=0.2,      # 가로 이동 범위
    height_shift_range=0.2,     # 세로 이동 범위
    shear_range=0.2,            # 기울이기 범위
    horizontal_flip=True,       # 좌우 반전
    fill_mode='constant',cval=0,         # 빈 부분 채우기
)

image_datagen = ImageDataGenerator(**data_gen_args)
label_datagen = ImageDataGenerator(**data_gen_args)

In [4]:
# 랜덤 색 변환 함수
def random_color_transform(image, hue_shift, sat_shift, val_shift):
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)  # BGR -> HSV
    hsv_image[..., 0] = (hsv_image[..., 0] + hue_shift) % 180  # Hue 변경
    hsv_image[..., 1] = np.clip(hsv_image[..., 1] * sat_shift, 0, 255)  # Saturation 변경
    hsv_image[..., 2] = np.clip(hsv_image[..., 2] * val_shift, 0, 255)  # Brightness 변경
    return cv2.cvtColor(hsv_image, cv2.COLOR_HSV2BGR)  # HSV -> BGR

In [11]:

# 경로 설정

for item in data_list:
  input_path = os.path.join(data_path,item,"image")
  label_path = os.path.join(data_path,item,"label")
  output_path_inputs = os.path.join(data_path,"augmented",item,"image")
  output_path_labels = os.path.join(data_path,"augmented",item,"label")

  os.makedirs(output_path_inputs, exist_ok=True)
  os.makedirs(output_path_labels, exist_ok=True)

  # 파일 로드
  input_files = sorted([os.path.join(input_path, f) for f in os.listdir(input_path) if f.endswith('.jpg')])
  label_files = sorted([os.path.join(label_path, f) for f in os.listdir(label_path) if f.endswith('.jpg')])

  # 증폭 시작
  num_augmentations_per_image = 10  # 각 이미지당 생성할 데이터 수

  for idx, (input_file, label_file) in enumerate(zip(input_files, label_files), start=0):
      input_img = cv2.imread(input_file)
      label_img = cv2.imread(label_file)
      input_img = np.expand_dims(input_img, 0)
      label_img = np.expand_dims(label_img, 0)

      input_gen = image_datagen.flow(input_img, batch_size=1, seed=42)
      label_gen = label_datagen.flow(label_img, batch_size=1, seed=42)

      for aug_idx in range(0, num_augmentations_per_image):
          aug_input = input_gen.next()[0].astype(np.uint8)
          aug_label = label_gen.next()[0].astype(np.uint8)

          # 색 변환
          hue_shift = np.random.randint(0, 180)
          sat_shift = np.random.uniform(0.5, 1.5)
          val_shift = np.random.uniform(0.5, 1.5)

          aug_input = random_color_transform(aug_input, hue_shift, sat_shift, val_shift)
          # aug_label = random_color_transform(aug_label, hue_shift, sat_shift, val_shift)

          # input_output_file = os.path.join(output_path_inputs, f'augmented_{idx}_{aug_idx}.jpg')
          # label_output_file = os.path.join(output_path_labels, f'augmented_{idx}_{aug_idx}.jpg')
          img_num=idx*10+aug_idx+1
          input_output_file = os.path.join(output_path_inputs, f'{img_num}.jpg')
          label_output_file = os.path.join(output_path_labels, f'annotated_{img_num}.jpg')

          cv2.imwrite(input_output_file, aug_input)
          cv2.imwrite(label_output_file, aug_label)

  print(f"총 {len(input_files) * num_augmentations_per_image}개의 데이터 증폭 완료")


총 720개의 데이터 증폭 완료
총 90개의 데이터 증폭 완료
총 230개의 데이터 증폭 완료


In [ ]:
data_path='./ajour' #여기서 augmentation만들 datapath 다시 설정
input_path = os.path.join(data_path,"image")
label_path = os.path.join(data_path,"label")
output_path_inputs = os.path.join(data_path,"augmented","image")
output_path_labels = os.path.join(data_path,"augmented","label")

os.makedirs(output_path_inputs, exist_ok=True)
os.makedirs(output_path_labels, exist_ok=True)

# 파일 로드
input_files = sorted([os.path.join(input_path, f) for f in os.listdir(input_path) if f.endswith('.jpg')])
label_files = sorted([os.path.join(label_path, f) for f in os.listdir(label_path) if f.endswith('.jpg')])

# 증폭 시작
num_augmentations_per_image = 10  # 각 이미지당 생성할 데이터 수

for idx, (input_file, label_file) in enumerate(zip(input_files, label_files), start=0):
    input_img = cv2.imread(input_file)
    label_img = cv2.imread(label_file)
    input_img = np.expand_dims(input_img, 0)
    label_img = np.expand_dims(label_img, 0)

    input_gen = image_datagen.flow(input_img, batch_size=1, seed=42)
    label_gen = label_datagen.flow(label_img, batch_size=1, seed=42)

    for aug_idx in range(0, num_augmentations_per_image):
        aug_input = input_gen.next()[0].astype(np.uint8)
        aug_label = label_gen.next()[0].astype(np.uint8)

        # 색 변환
        hue_shift = np.random.randint(0, 180)
        sat_shift = np.random.uniform(0.5, 1.5)
        val_shift = np.random.uniform(0.5, 1.5)

        aug_input = random_color_transform(aug_input, hue_shift, sat_shift, val_shift)
        # aug_label = random_color_transform(aug_label, hue_shift, sat_shift, val_shift)

        # input_output_file = os.path.join(output_path_inputs, f'augmented_{idx}_{aug_idx}.jpg')
        # label_output_file = os.path.join(output_path_labels, f'augmented_{idx}_{aug_idx}.jpg')
        img_num=idx*10+aug_idx+711 #710대신 시작할 번호 넣으면 됨(train기준준)
        input_output_file = os.path.join(output_path_inputs, f'{img_num}.jpg')
        label_output_file = os.path.join(output_path_labels, f'annotated_{img_num}.jpg')

        cv2.imwrite(input_output_file, aug_input)
        cv2.imwrite(label_output_file, aug_label)

print(f"총 {len(input_files) * num_augmentations_per_image}개의 데이터 증폭 완료")

총 1340개의 데이터 증폭 완료
